In [1]:
# 필요 라이브러리 호출
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
df = pd.read_csv('data/naver_comment.csv', encoding='utf8')
df.head()

,title,score,score_num,best_num,best,best_recomm,best_unrecomm,comment_num,comment,star
0,엑스트라 데이즈,9.62,101,6,아 근데 실제로 저런 기술이 있다면 쫌 싸할것같다.. 내가 죽어도 사람들은 나랑 똑...,126,0,42,버키 왜 하나만 블러예욬ㅋㅋㅋㅋㅋ\n이 웹툰은 개재밌을거라고 나한테 텔레파시가왔다\...,9.65
1,원수를 사랑하라,9.95,31400,15,근데 여주 대단하다...엄마랑 아빠가 저런 사람들인데 멀끔히 차려입고 면접보는 거면...,129323,3789,984,그림체 마음에 드네 \n그림체 미친\n면접관중에 존나 보기싫은얼굴이보이는데?\n누가...,9.92
2,우투리,8.50,159,4,"학원액션물 만화계의 거장 임재원님의 2번째 작품 우투리입니다.... 스토리, 그림체...",194,5,51,진짜 퀄리티 말도 안되는데 덴마 이후로 미완결 작품은 그냥 혐오스럽다\n우와아아아!...,9.20
3,롭플롭,9.78,290,5,"나 이 작가 작품 좋아하네.... 잭슨의 관, 데빌샷 보고 바로 들어옴 \n잭슨의 ...",410,2,64,작가님 어케 이렇게 그천이신거에요\n노랑머리 친구 명암때문인진 몰라도장례식 이후에 ...,9.91
4,약초마을 연쇄살초사건,9.73,10226,15,아니ㅋㅋㅋㅋㅋㅋ작가님ㅋㅋㅋㅋㅋㅋ이제 작물 쪽으로 길을 트신건가요ㅋㅋㅋㅋㅋㅋㅋㅋ\n팀...,62851,254,1463,ㅋㅋㅋㅋㅋㅋㅋㅋ\n조낸 무서운 이야기인데 약초란 사실과 그림체 때문에 잔혹함이 전달...,9.87


In [3]:
okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, stem=True)
    return tokens_ko

import re

def hangul(text):
    return re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", text)

In [9]:
def train_to_evaluate(X, y, min_df, max_df):
    tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1, 2), min_df=min_df, max_df=max_df)
    tfidf_vect.fit(X)
    tfidf_matrix = tfidf_vect.transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    
    lgbm = LGBMRegressor()
    xgb = XGBRegressor()
    rf = RandomForestRegressor()

    regs = [lgbm, xgb, rf]

    result = []
    for reg in regs:
        reg.fit(X_train, y_train)
        pred = reg.predict(X_val)
        mae = mean_absolute_error(y_val, pred)
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        print(reg.__class__.__name__)
        print(f'MAE : {mae:.4f}, RMSE : {rmse:.4f}')
        
    return regs

# 1. 베댓 + 최근 댓글 15개 통합

## 1-1. `min_df=0, max_df=0.9`, 특수문자도 함께 처리

In [10]:
%%time

X = df['best'] + '\n' + df['comment']
y = df['star']

regs1_1 = train_to_evaluate(X, y, 0, 0.9)

LGBMRegressor
MAE : 0.2664, RMSE : 0.4567
XGBRegressor
MAE : 0.2510, RMSE : 0.4586
RandomForestRegressor
MAE : 0.2563, RMSE : 0.4783
Wall time: 3min 24s


## 1-2. `min_df=3, max_df=0.95`, 특수문자도 함께 처리

In [13]:
%%time

regs1_2 = train_to_evaluate(X, y, 3, 0.95)

LGBMRegressor
MAE : 0.2668, RMSE : 0.4658
XGBRegressor
MAE : 0.2450, RMSE : 0.4812
RandomForestRegressor
MAE : 0.2514, RMSE : 0.4639
Wall time: 1min 43s


## 1-3. `min_df=0, max_df=0.9`, 특수문자 제거

In [11]:
%%time

regs1_3 = train_to_evaluate(X.apply(hangul), y, 0, 0.9)

LGBMRegressor
MAE : 0.2727, RMSE : 0.4532
XGBRegressor
MAE : 0.2819, RMSE : 0.5556
RandomForestRegressor
MAE : 0.2601, RMSE : 0.4723
Wall time: 3min 14s


## 1-4. `min_df=3, max_df=0.95`, 특수문자 제거

In [12]:
%%time

regs1_4 = train_to_evaluate(X.apply(hangul), y, 3, 0.95)

LGBMRegressor
MAE : 0.2606, RMSE : 0.4384
XGBRegressor
MAE : 0.3035, RMSE : 0.5792
RandomForestRegressor
MAE : 0.2492, RMSE : 0.4621
Wall time: 1min 46s


# 2. 최근 댓글 15개로만

## 2-1. `min_df=0, max_df=0.9`, 특수문자도 함께 처리

In [19]:
%%time

X = df['comment']
y = df['star']

regs2_1 = train_to_evaluate(X, y, 0, 0.9)

LGBMRegressor
MAE : 0.2368, RMSE : 0.4019
XGBRegressor
MAE : 0.2504, RMSE : 0.4881
RandomForestRegressor
MAE : 0.2195, RMSE : 0.4376
Wall time: 1min 35s


## 2-2. `min_df=3, max_df=0.95`, 특수문자도 함께 처리

In [20]:
regs2_2 = train_to_evaluate(X, y, 3, 0.95)

LGBMRegressor
MAE : 0.2485, RMSE : 0.4137
XGBRegressor
MAE : 0.2475, RMSE : 0.4764
RandomForestRegressor
MAE : 0.2286, RMSE : 0.4420


## 2-3. `min_df=0, max_df=0.9`, 특수문자 제거

In [21]:
%%time

regs2_3 = train_to_evaluate(X.apply(hangul), y, 0, 0.9)

LGBMRegressor
MAE : 0.2411, RMSE : 0.4102
XGBRegressor
MAE : 0.2517, RMSE : 0.4773
RandomForestRegressor
MAE : 0.2288, RMSE : 0.4523
Wall time: 1min 34s


## 2-4. `min_df=3, max_df=0.95`, 특수문자 제거

In [22]:
%%time

regs2_4 = train_to_evaluate(X.apply(hangul), y, 3, 0.95)

LGBMRegressor
MAE : 0.2466, RMSE : 0.4182
XGBRegressor
MAE : 0.2376, RMSE : 0.4372
RandomForestRegressor
MAE : 0.2215, RMSE : 0.4383
Wall time: 58.8 s


- 특수문자 제거를 하지 않는 것이 더 높은 예측력(아주 약간)을 가짐
- `min_df`와 `max_df`는 뭘로 하든지 비슷하다
- TF-IDF로 예측 결과 MAE는 0.22\~0.30, RMSE는 0.40\~0.55